In [2]:
import pandas as pd
import re

# Treatment of the autism_treatment_centers_bookimed.csv:

In [ ]:
# Load your data into a pandas DataFrame
df = pd.read_csv("autism_treatment_centers_bookimed.csv")
df.info()

- split the country attribut to city and country 

In [ ]:

# Define a function to split the country attribute
def split_country(text):
  # Split by comma, assuming the city is before the comma
  parts = text.split(",")
  country = parts[0].strip()
  city = ",".join(parts[1:]).strip()
  return city, country

# Apply the function to the "country" column and create new columns
df[["city", "country"]] = df["country"].apply(split_country).apply(pd.Series)

# Print the modified DataFrame
print(df)


In [ ]:
df.head()

- Specific treatment to eliminate any conflict information

In [ ]:
# Filter rows where country is not 'Israel'
df = df[df['country'] != 'Israel']

df.head()

 - standardizing abbreviations  and Normalize address 

In [ ]:
import re

def normalize_address(address):
   
    
    # Normalize street names and types
    abbreviations = {
        "St": "Street",
        "Ave": "Avenue",
        "Rd": "Road",
        "Blvd": "Boulevard",
        "Dr": "Drive",
        "Hosp": "Hospital",
        "NP": "Neurology Psychiatry",
        "No:": "Number :" ,# Assuming "NO" stands for "Number"
        "CD" : "Clinic Department " ,
        "D:" : "Door : ",
        "Cd" : "Cadde",
        "LV" : "Latvia"

    }
    
    # Iterate over each abbreviation and replace it in the address
    for abbreviation, full_word in abbreviations.items():
        address = re.sub(r'\b' + re.escape(abbreviation) + r'\b', full_word, address)
    

    
    # Return the normalized address
    return address.strip()

# Apply normalization to the "address" column
df['normalized_address'] = df['address'].apply(normalize_address)
print(df['normalized_address'])



In [ ]:
df.head()

- reorder attributes and select desired attributes

In [ ]:
# Define the desired order of columns
desired_columns = ['name', 'city' , 'country', 'normalized_address', 'latitude', 'longitude' , 'info', 'free_quote_link']

# Reorganize the columns and select only the necessary columns
df = df.reindex(columns=desired_columns)

# Rename the 'normalized_address' column to 'address'
df = df.rename(columns={'normalized_address': 'address'})

# Print the updated DataFrame
df.head()


In [ ]:
df.drop_duplicates(inplace=True)
df.info()

In [ ]:
# Exporter le DataFrame vers un fichier CSV
df.to_csv('autism_treatment_centers_bookimed_processed.csv', index=False)

# Treatment of autism_treatment_centers_placidway.csv:

In [ ]:
# Load your data into a pandas DataFrame
dp = pd.read_csv("autism_treatment_centers_placidway.csv")

In [ ]:
dp.head()

In [ ]:
dp.info()

- Remove Duplicates:

In [ ]:
dp.drop_duplicates(inplace=True)

- Determine the city 

In [ ]:
# Assuming the address contains the city information separated by comma
dp['city'] = dp['address'].apply(lambda x: x.split(',')[-2].strip())
dp.head(17)

- standardize the addresses

In [ ]:
import re

def standardize_address(address, city):
    # Normalize street names and types
    abbreviations = {
        "St.": "Street",
        "Ave.": "Avenue",
        "Rd.": "Road",
        "Blvd.": "Boulevard",
        "Dr.": "Drive",
        "Ste.": "Suite",
        "Fl.": "Floor",
        "No.": "Number",
        "Bldg.": "Building",
        "Apt.": "Apartment",
        "Pl.": "Place",
        "Ct.": "Court"
    }
    
    # Iterate over each abbreviation and replace it in the address
    for abbreviation, full_word in abbreviations.items():
        address = re.sub(r'\b' + re.escape(abbreviation) + r'\b', full_word, address)
    
    
    return address.strip()

# Apply address standardization to the "address" column
dp['standardized_address'] = dp.apply(lambda x: standardize_address(x['address'], x['city']), axis=1)

# Display the standardized addresses
print(dp['standardized_address'])

- reorder attributes and select desired attributes

In [ ]:
# Define the desired order of columns
desired_columns = ['second_name', 'city' , 'country', 'standardized_address', 'latitude', 'longitude']

# Reorganize the columns and select only the necessary columns
dp = dp.reindex(columns=desired_columns)

# Rename the 'normalized_address' column to 'address'
dp = dp.rename(columns={'standardized_address': 'address'})
dp = dp.rename(columns={'second_name': 'name'})

# Print the updated DataFrame
dp.head()


In [ ]:
# Exporter le DataFrame vers un fichier CSV
dp.to_csv('autism_treatment_centers_placidway_processed.csv', index=False)

# Treatment of the autism_traitement_centers_Algeria.csv : 

In [ ]:
dA = pd.read_csv("autism_traitement_centers_Algeria.csv")
dA.head()

In [ ]:

# Define the desired order of columns
desired_columns = ['name', 'city', 'country', 'address', 'latitude', 'longitude', 'info']


# Reorganize the columns and select only the necessary columns
dA = dA.reindex(columns=desired_columns)

# Print the updated DataFrame
dA.head()

In [ ]:
# Exporter le DataFrame vers un fichier CSV
dA.to_csv('autism_traitement_centers_Algeria_processed.csv', index=False)

# Processing of the autism_treatment_centers_autism_now.csv file:

In [2]:
dan = pd.read_csv("autism_treatment_centers_autism_now.csv")
dan.head()

,name,state,city,county,contact info
0,Alabama Autism,Alabama,NaN,NaN,By Website
1,Arc Fayette Lamar,Alabama,Fayette,Fayette,Arc Fayette Lamar\r\n1428 Temple Ave S\r\nFaye...
2,Arc of The Chattahoochee Valley,Alabama,Valley,Chambers,Arc of The Chattahoochee Valley\r\nPO Box 416\...
3,ARC- Baldwin County,Alabama,Loxley,Baldwin,"ARC- Baldwin County\r\nPO Box 400\r\nLoxley, A..."
4,Athens/Limestone Arc,Alabama,Athens,Limestone,Athens/Limestone Arc\r\n427 Rogers Street\r\nA...


In [3]:
# Fix errors in some "contact info" values
dan['contact info'] = dan['contact info'].str.replace(' or By Website', '')
dan['contact info'] = dan['contact info'].replace('By Website', None)
dan['contact info'] = dan['contact info'].str.replace('770-904- 4474', '770-904-4474')

In [4]:
# Remove the name from the contact info
def remove_name_from_contact(contact_info, name):
    contact_info = contact_info.replace(name, "").strip() if contact_info is not None else None
    return contact_info

dan['contact info'] = dan.apply(lambda x: remove_name_from_contact(x['contact info'], x['name']), axis=1)

dan.head()

,name,state,city,county,contact info
0,Alabama Autism,Alabama,NaN,NaN,None
1,Arc Fayette Lamar,Alabama,Fayette,Fayette,"1428 Temple Ave S\r\nFayette, AL 35555-3111\r\..."
2,Arc of The Chattahoochee Valley,Alabama,Valley,Chambers,"PO Box 416\r\nValley, AL 36854-0416\r\nvalleyh..."
3,ARC- Baldwin County,Alabama,Loxley,Baldwin,"PO Box 400\r\nLoxley, AL 36651\r\njsp36@bellso..."
4,Athens/Limestone Arc,Alabama,Athens,Limestone,"427 Rogers Street\r\nAthens, AL 35611\r\ntwsai..."


In [5]:
# Move phone numbers and emails to a new "info" column
def extract_phone_email(row, contact_info):
    if contact_info is None:
        return row
    regex = r'[\w\.-]+@[\w\.-]+\.\w+|\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}'
    phone_email = re.findall(regex, contact_info)
    row['info'] = ", ".join(phone_email)
    row['contact info'] = re.sub(regex, "", contact_info).strip()
    return row

dan = dan.apply(lambda x: extract_phone_email(x, x['contact info']), axis=1)
dan['contact info'] = dan['contact info'].str.replace("\r\n", ", ").replace("\n", ", ").str.strip()

dan.head()

,city,contact info,county,info,name,state
0,NaN,None,NaN,NaN,Alabama Autism,Alabama
1,Fayette,"1428 Temple Ave S, Fayette, AL 35555-3111",Fayette,(205) 932-8642,Arc Fayette Lamar,Alabama
2,Valley,"PO Box 416, Valley, AL 36854-0416",Chambers,"valleyhaven@charterinternet.com, (334) 756-2868",Arc of The Chattahoochee Valley,Alabama
3,Loxley,"PO Box 400, Loxley, AL 36651",Baldwin,"jsp36@bellsouth.net, (251) 964-4451",ARC- Baldwin County,Alabama
4,Athens,"427 Rogers Street, Athens, AL 35611",Limestone,"twsaint@pcinet.net, (256) 233-2999",Athens/Limestone Arc,Alabama


In [6]:
# Fix errors in some "address" values
dan['city'] = dan['city'].str.replace('San Bernadino', 'San Bernardino')
dan['contact info'] = dan['contact info'].str.replace(', California', ', CA')
dan['contact info'] = dan['contact info'].str.replace(', Grand Jct,', ', Grand Junction,')
dan['contact info'] = dan['contact info'].str.replace(', Colorado Spgs,', ', Colorado Springs,')
dan['contact info'] = dan['contact info'].str.replace(', Ft Walton Bch,', ', Fort Walton Beach,')
dan['contact info'] = dan['contact info'].str.replace(', Pt Charlotte,', ', Port Charlotte,')
dan['contact info'] = dan['contact info'].str.replace(', St Augustine,', ', St. Augustine,')
dan['contact info'] = dan['contact info'].str.replace(', Ft Lauderdale,', ', Fort Lauderdale,')
dan['contact info'] = dan['contact info'].str.replace(', Defuniak Spgs,', ', Defuniak Springs,')
dan['contact info'] = dan['contact info'].str.replace(', Chicago Hts,', ', Chicago Heights,')
dan['contact info'] = dan['contact info'].str.replace(', Donaldsonvlle,', ', Donaldsonville,')
dan['contact info'] = dan['contact info'].str.replace(', St Martinvlle,', ', St. Martinville,')
dan['contact info'] = dan['contact info'].str.replace(', N Weymouth,', ', North Weymouth,')
dan['contact info'] = dan['contact info'].str.replace(', Clinton Twp,', ', Clinton Township,')
dan['contact info'] = dan['contact info'].str.replace(', Mt Pleasant,', ', Mount Pleasant,')
dan['contact info'] = dan['contact info'].str.replace(', Saint Paul,', ', St. Paul,')
dan['contact info'] = dan['contact info'].str.replace(', Saint Cloud,', ', St. Cloud,')
dan['contact info'] = dan['contact info'].str.replace(', Saint Louis,', ', St. Louis,')
dan['contact info'] = dan['contact info'].str.replace(', N Brunswick,', ', North Brunswick,')
dan['city'] = dan['city'].str.replace('Mounty Airy', 'Mount Airy')
dan['contact info'] = dan['contact info'].str.replace(', Winston Salem,', ', Winston-Salem,')
dan['contact info'] = dan['contact info'].str.replace(', N Providence,', ', North Providence,')
dan['contact info'] = dan['contact info'].str.replace(', Fredericksbrg,', ', Fredericksburg,')
dan['contact info'] = dan['contact info'].str.replace(', St George,', ', St. George,')
dan['city'] = dan['city'].str.replace('Saint George', 'St. George')
dan['contact info'] = dan['contact info'].str.replace(', Richland Ctr,', ', Richland Center,')

In [7]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [8]:
# rename "contact info" to "address"
dan = dan.rename(columns={'contact info': 'address'})

# Fix the "address" column format
def fix_address_format(state: str, county: str, city: str, address: str):
    if not address:
        return None
    address_elements = address.split(", ")
    correct_address_list = []
    # check if the first element is a PO box
    if "PO Box" in address_elements[0]:
        correct_address_list.append(address_elements.pop(0).split(" ")[2])
    else:
        correct_address_list.append("")

    if address_elements[0].split(" ")[0].isdigit():
        st_number, st_name = address_elements.pop(0).split(" ", 1)
        correct_address_list.extend([st_name, st_number])
    else:
        correct_address_list.extend(["", ""])

    # check if the third element is the same as the city
    while len(address_elements) > 0 and address_elements[0] != city:
        address_elements.pop(0)
    if len(address_elements) == 0:
        print(f"City '{city}' not found in address '{address}'\nCurrent elements: {address_elements}\nCorrect address list: {correct_address_list}")
        return None
    else:
        correct_address_list.append(address_elements.pop(0))

    # check if the fourth element is of the form "state code zip"
    while len(address_elements) > 0 and address_elements[0].split(" ")[0] != us_state_to_abbrev[state]:
        address_elements.pop(0)
    if len(address_elements) == 0:
        print(f"State '{state}' not found in address '{address}'\nCurrent elements: {address_elements}\nCorrect address list: {correct_address_list}")
        return None
    else:
        correct_address_list.append(address_elements.pop(0).split(" ")[1])

    # add county at the end
    correct_address_list.append(county)

    return ",".join(correct_address_list).replace(", ,", ",,").strip()

# Apply the function to the "address" column and create new columns
dan['address'] = dan.apply(lambda x: fix_address_format(x['state'], x['county'], x['city'], x['address']), axis=1)

# Print the modified DataFrame
dan.head()

City 'Overland Park' not found in address 'Ext 209'
Current elements: []
Correct address list: ['', '', '']
City 'Columbus' not found in address '1-'
Current elements: []
Correct address list: ['', '', '']
City 'Oklahoma City' not found in address 'ext. 47123'
Current elements: []
Correct address list: ['', '', '']
City 'West Columbia' not found in address 'ext 106'
Current elements: []
Correct address list: ['', '', '']
City 'Richmond' not found in address 'ext 101'
Current elements: []
Correct address list: ['', '', '']
City 'Madison' not found in address 'ext 2'
Current elements: []
Correct address list: ['', '', '']


,city,address,county,info,name,state
0,NaN,None,NaN,NaN,Alabama Autism,Alabama
1,Fayette,",Temple Ave S,1428,Fayette,35555-3111,Fayette",Fayette,(205) 932-8642,Arc Fayette Lamar,Alabama
2,Valley,"416,,,Valley,36854-0416,Chambers",Chambers,"valleyhaven@charterinternet.com, (334) 756-2868",Arc of The Chattahoochee Valley,Alabama
3,Loxley,"400,,,Loxley,36651,Baldwin",Baldwin,"jsp36@bellsouth.net, (251) 964-4451",ARC- Baldwin County,Alabama
4,Athens,",Rogers Street,427,Athens,35611,Limestone",Limestone,"twsaint@pcinet.net, (256) 233-2999",Athens/Limestone Arc,Alabama


In [9]:
# Add missing columns
dan['country'] = "USA"
dan['latitude'] = None
dan['longitude'] = None

# Define the desired order of columns
desired_columns = ['name', 'country', 'state', 'address', 'latitude', 'longitude', 'info']

# Reorganize the columns
dan = dan.reindex(columns=desired_columns)

# Rename "state" to "city"
dan = dan.rename(columns={'state': 'city'})

# Remove rows with empty address
dan = dan[dan['address'].notna()]

# Print the updated DataFrame
dan.head()

,name,country,city,address,latitude,longitude,info
1,Arc Fayette Lamar,USA,Alabama,",Temple Ave S,1428,Fayette,35555-3111,Fayette",None,None,(205) 932-8642
2,Arc of The Chattahoochee Valley,USA,Alabama,"416,,,Valley,36854-0416,Chambers",None,None,"valleyhaven@charterinternet.com, (334) 756-2868"
3,ARC- Baldwin County,USA,Alabama,"400,,,Loxley,36651,Baldwin",None,None,"jsp36@bellsouth.net, (251) 964-4451"
4,Athens/Limestone Arc,USA,Alabama,",Rogers Street,427,Athens,35611,Limestone",None,None,"twsaint@pcinet.net, (256) 233-2999"
6,Autism Society of Alabama,USA,Alabama,",Dolly Ridge Road,4217,Birmingham,35243,Jeffer...",None,None,877-428-8476


In [10]:
dan.to_csv('autism_treatment_centers_autism_now_processed.csv', index=False)

# Merging and Exporting Multiple Dataframes into a Single CSV File

In [ ]:

# Charger les fichiers CSV
df1 = pd.read_csv('autism_treatment_centers_placidway_processed.csv')
df2 = pd.read_csv('autism_treatment_centers_bookimed_processed.csv')
df3 = pd.read_csv('autism_traitement_centers_Algeria_processed.csv')



In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df3.head()

In [ ]:
merged_df = pd.concat([df1, df2, df3], ignore_index=True)
merged_df.head(30)


In [ ]:
merged_df.to_csv('autism_center.csv', index=False)

In [6]:
final_d = pd.read_csv('autism_center_b.csv')
d4 = pd.read_csv('autism_treatment_centers_autism_now_processed.csv')

# reorder the columns "city" and "country" to "country" and "city"
final_d = final_d[['name', 'country', 'city', 'address', 'latitude', 'longitude', 'info', 'free_quote_link']]

merged_df = pd.concat([final_d, d4], ignore_index=True)
merged_df.to_csv('autism_center.csv', index=False)